In [1]:
import asyncio
import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from pydantic import BaseModel

from typing import List
from typing import Optional
from typing import Tuple
from typing import Union

from bs4 import BeautifulSoup


In [31]:
from era_ticket_crawler import EraTicketCrawler
from crawler_base import SeatArea
from crawler_base import ShowTime
from crawler_base import Activity

In [37]:
crawler = EraTicketCrawler(
    urls=[
        # "https://ticket.com.tw/application/UTK02/UTK0201_00.aspx?PRODUCT_ID=P0DXG163",
        "https://ticket.com.tw/application/UTK02/UTK0201_00.aspx?PRODUCT_ID=P0CNUP9W",  # 取消演出
        # "https://ticket.com.tw/application/UTK02/UTK0201_00.aspx?PRODUCT_ID=P0C9W4PE",
    ]
)

In [38]:
soups = crawler.get_soups(crawler.urls)

In [39]:
soup = soups[0]
#soup

In [40]:

title = soup.find("h1", {"align": "center"}).find("span", {"class": "title"}).text
info_body = soup.find("tbody")
show_times = []
for i_sess, sess in enumerate(info_body.find_all("tr")):
    sess_info = sess.find_all("td")
    # Parse date from <time class="icon"> in first <td>
    sess_date = sess_info[0].find("time", {"class": "icon"})["datetime"].strip()
    sess_time = sess_info[0].find("span", {"class": "time"}).text.strip()
    all_prices = sess_info[2].find("span").text.strip().split("、")
    sold_out_prices = sess_info[2].find("span").find_all("del")
    sold_out_prices = list(map(lambda x: x.text.strip(), sold_out_prices))
    is_cancelled = "取消" in sess_info[3].find("button").text.strip()
    print(sess_date, sess_time, sold_out_prices, all_prices, is_cancelled)
    seat_areas = []
    for i_price, price in enumerate(all_prices):
        seat_areas.append(SeatArea(
            id_ = i_price,
            name = str(price),
            available = price not in sold_out_prices and not is_cancelled))
    show_times.append(ShowTime(
        id_ = i_sess,
        name = f"{sess_date} {sess_time}",
        datetime = f"{sess_date} {sess_time}",
        seat_areas = seat_areas,
        available = any([area.available for area in seat_areas])))

2024-02-21 14:00 [] ['2800', '1800'] True
2024-02-21 19:00 [] ['2800', '1800'] True
